In [ ]:
# Uczenie głębokie z wykorzystaniem dedykowanych narzędzi

In [ ]:
# Outline:
    # 1) intro do deep learning software
    
    # 2) intro do obliczeń symbolicznych
        # 2.1) ćwiczenie
        
    # 3.) keras hands-on
        # 3.0) intro do kerasa
        # 3.1) ćwiczenie: implementacja modelu
        # 3.2) implementacja warstw
        # 3.3) ćwiczenie: implementacja własnej warstwy
    
    # 4.) hands-on part II: prezentacja i ćwiczenie z rozponawania obrazu

In [11]:
# W czym pisze się sieci neuronowe
    # dedykowane narzędzia posiadają zoptymalizowane operacje matematyczne
    # pozwalają także na wykonywanie obliczeń równolegle lub z użyciem GPU bez wkładu programisty
    # ułatwiają konstruckję i uczenie najpopularniejszych modeli. 

In [ ]:
# Deep Learning landscape: styczeń 2017
    # Theano: do niedawna najpopularniejszy framework, univ. of Montreal, jest kompilatorem wyrażeń mat. 
    # Torch:  do niedawna jedyne zagrożenie dla theano, rozwijany przez fb oraz twittera
    # Tensorflow: nowa biblioteka od Google. Najpopularniejszy i najlepiej rozwinięty
    
    # TODO: Linki, informacje o popularności, odrobinę więcej opisu

In [1]:
import tensorflow as tf

In [4]:
# Obliczenia symboliczne

# Sercem większości frameworków wysokiego poziomu jest wyspecjalizowany backend obliczeniowy.
# Keras, którego będziemy dzisiaj używać, posiada dwa takie backendy: Tensorflow oraz Theano
# Warto poświęcić chwilę, żeby zapoznać się z najbardziej podstawowymi zastadami ich działania.

In [ ]:
# Tensorflow i Theano to przykłady narzędzi, które tworzą graf operacji symbolicznych.
# Oznacza to, że operacje wykonywane na zmiennych nie mają natychmiastowego efektu.
# Są jedynie dodawane do grafu
# Następnie graf jest kompilowany i wykonwyany

In [20]:
# przykład
import tensorflow as tf

a = tf.Variable(2.)
b = tf.Variable(2.)
result = a + b

print(result)

# result nie jest równe 4! Jest tylko symbolem, oznaczającym "wynik dodania zmiennej 'a' do zmiennej 'b'"

Tensor("add_2:0", shape=(), dtype=float32)


In [ ]:
# Tensorflow pozwala wizualizować stworzony graf:
print_default_tf_graph()

In [5]:
# Przepływ danych w grafie możesz zainicjalizować w następujący sposób:
execute_tensorflow_graph(outputs=[result])

[4.0]


In [21]:
# Powyższy kod był tylko demonstracją idei obliczeń symbolicznych. 
# W praktyce nie znamy początkowych wartości a i b. 
# Jak napisać w tensorflow odpowiednik poniższego kodu?

def add(a, b):
    return a + b

result = add(2, 2)

In [ ]:
# służą do tego "placeholdery'.
# Informujemy tensorflow, jakiego rodzaju zmiennych ma się spodziewać, ale nie mówimy, jakie są ich wartości

a = tf.placeholder()
b = tf.placeholder()

result = a + b 

# Aby zainicjalizować przepływ danych w grafie, należy podać początkowe wartości parametrów:
inputs = {
    a: 2.,
    b: 2.
}
execute_tensorflow_graph(outputs=[result], inputs=inputs)

In [15]:
# zadanie: oblicz kwadraty liczb 2, 3 i 4 używając tensorflow
    # wykorzystaj placeholdery

In [ ]:
# rozwiązanie
x = tf.placeholder()
result = x * x

for i in range(2, 5):
    inputs = {x: i}
    r = execute_tensorflow_graph(outputs=[result], inputs=inputs)
    
    print(r)

In [ ]:
# zadanie: dodaj dwie macierze jednostkowe 2x2

In [ ]:
# rozwiązanie

A = tf.placeholder()
B = tf.placeholder()
result = A + B

inputs = {
    A: np.array([
            [1, 0],
            [0, 1]
        ]),
    B: np.array([
            [1, 0],
            [0, 1]
        ])
}

execute_tensorflow_graph(outputs=[result], inputs=inputs)

In [22]:
# Jakie są zalety tego dziwnego podejścia?
    # Optymalizacja: znając cały graf, kompilator może zoptymalizować wykonaywanie operacji
    # Współbieżność: kompilator sam wykorzysta OpenMP aby wykonać obliczenia równolegle
    # Niezależność od architektury: Znając graf, kompilator może wygenerować kod dla CPU / GPU / FPGA etc.


In [17]:
# Wyższy poziom abstrakcji

# Warto wiedzieć, jakie są ogólne zasady rządzące tak popularnymi narzędziami jak tensorflow 
# W praktyce jednak, najczęściej korzystamy z narzędzi wyższego poziomu, które pozwalają nie myśleć o obliczeniach symbolicznych.

In [18]:
# Keras
    # Jednym z popularniejszych narzędzi jest Keras. 
    # Pozwala on łatwo tworzyć modele z popularnych "klocków"

In [23]:
# Aby z powodzeniem zastosować algorytm uczenia głębokiego 
# Potrzebne nam będą 4 elementy:
    # dane
    # architektura sieci
    # funckja straty
    # algorytm optymalizacji

In [ ]:
# Dane

# keras współpracuje z numpy, w związku z tym te etap nie będzie dla nas problemem
# wystarczy wczytać dane znane już z poprzednich zajęć:
X = np.load(...)
y = np.load(...)

# warto zaznaczyć, iż keras potrafi współpracować także z danymi w innym formacie, np. hdf5

In [ ]:
# Architektura sieci. 

# Keras wspiera dwie metody definiowania architektury: model liniowy i API funkcjonalne.

In [ ]:
# [pozbyć się tego kodu...?]

# Przykładowy model może wyglądać następująco:

# Liniowo:
model = Sequential()
model.add(Dense(32, input_dim=128))
model.add(Activation('relu'))

# Funckojnalnie:
x = Input(input_dim=128)
x = Dense(32)(x)
x = Activation('relu')(x)
model = Model(x)

In [26]:
# Skupimy się dzisiaj głównie na modelach liniowych, ponieważ w zupełności wystarczają one do większośći zastosowań praktycznych.

In [ ]:
# Warstwy:

# Podstawowym elementem, z którego budowana jest sieć neuronowa, jest warstwa. 
# Model liniowy w kerasie to nic innego, jak tylko złożenie kolejnych, następujących po sobie warstw.

# Warstwa (layer) jest podstawową jednostką przetwarzania: 
# przyjmuje ona jakąś macierz (tensor) na wejściu, modyfikuje ją, a następnie podaje na wyściu

In [ ]:
# Aby zobrazować to w praktyce, zaimplementujemy regresję logistyczną, poznaną na poprzednich zajęciach, z użyciem kerasa

In [ ]:
# Regresja logisytczna 

In [ ]:
# ćwiczenie?

In [ ]:
# MLP

In [ ]:
# ćwiczenie?

In [ ]:
# Implementacja własnej warstwy

In [ ]:
# ćwiczenie?

In [ ]:
# Teraz, kiedy znamy już podstawowe zasady rządzące Kerasem
# Pokażemy, w jak łatwy sposób można wykorzystać to narzędzie to rozpoznawania obrazu.

# W sposób trywialnie prosty otrzymamy wynik, który jeszcze parę lat temu pozostawał poza zasięgiem najbardziej zaawansowanych metod.

In [9]:
o.name# Budowanie modelu

'Variable/initial_value'

In [4]:
# Implementacja warstw

In [5]:
# Sieci konwolucyjne

In [7]:
# Potęga Deep Learning

# zbuduj swój model, mając "core"